In [1]:
from pathlib import Path
import open3d as o3d
import copy
import numpy as np

In [2]:
def draw_registration_result_original_color(source, target, transformation, source_file):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    
    # 入力ファイル名の末尾に "_colored_icp" を追加して新しいファイル名を生成
    source_path = Path(source_file)  # 引数として受け取ったファイル名を使用
    new_filename = source_path.stem + "_colored_icp.ply"  # "_colored_icp" を追加
    new_path = source_path.with_name(new_filename)

    # 変換された点群を新しいファイル名で保存
    o3d.io.write_point_cloud(str(new_path), source_temp)
    print("save ",new_filename)
    # 点群を視覚化
    o3d.visualization.draw_geometries([source_temp, target],
                                      zoom=0.5,
                                      front=[-0.2458, -0.8088, 0.5342],
                                      lookat=[1.7745, 2.2305, 0.9787],
                                      up=[0.3109, -0.5878, -0.7468])


In [4]:

print("1. Load two point clouds and show initial pose")
source_path="output/0013/0013_transformed.ply"
target_path="output/0014/0014_transformed.ply"
# source_path="0005_norm.ply"
# target_path="0006_norm.ply"
source = o3d.io.read_point_cloud(source_path)
target = o3d.io.read_point_cloud(target_path)

# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result_original_color(source, target, current_transformation, source_path)

1. Load two point clouds and show initial pose
save  0013_transformed_colored_icp.ply


In [21]:
# point to point ICP
current_transformation = np.identity(4)
print("2. Point-to-point ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.2.")
result_icp = o3d.pipelines.registration.registration_icp(
    source, target, 0.3, current_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(result_icp)
draw_registration_result_original_color(source, target, result_icp.transformation, source_path)
icp_trans=result_icp.transformation


2. Point-to-point ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.2.
RegistrationResult with fitness=9.986592e-01, inlier_rmse=6.544929e-02, and correspondence_set size of 309095
Access transformation to get result.
save  0005_norm_colored_icp.ply


In [12]:
# Initial transformation
current_transformation = np.identity(4)

# Initial distance threshold
initial_threshold = 1
min_threshold = 0.0005  # Minimum threshold value
threshold_decrement = 0.05  # Amount to decrease the threshold
threshold = initial_threshold

print("Starting ICP with decreasing distance threshold...")
while threshold >= min_threshold:
    print(f"Trying with distance threshold: {threshold}")
    
    # Point-to-point ICP registration
    result_icp = o3d.pipelines.registration.registration_icp(
        source, target, threshold, current_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    
    print(result_icp)

    
    # Update the current transformation for the next iteration
    current_transformation = result_icp.transformation
    
    # Decrease the threshold
    threshold /= 2
draw_registration_result_original_color(source, target,
                                        result_icp.transformation, source_path)
print("Finished ICP with decreasing distance threshold.")

Starting ICP with decreasing distance threshold...
Trying with distance threshold: 1
RegistrationResult with fitness=1.000000e+00, inlier_rmse=5.362144e-02, and correspondence_set size of 235205
Access transformation to get result.
Trying with distance threshold: 0.5
RegistrationResult with fitness=9.999915e-01, inlier_rmse=5.358711e-02, and correspondence_set size of 235203
Access transformation to get result.
Trying with distance threshold: 0.25
RegistrationResult with fitness=9.999915e-01, inlier_rmse=5.358671e-02, and correspondence_set size of 235203
Access transformation to get result.
Trying with distance threshold: 0.125
RegistrationResult with fitness=9.207160e-01, inlier_rmse=3.427111e-02, and correspondence_set size of 216557
Access transformation to get result.
Trying with distance threshold: 0.0625
RegistrationResult with fitness=8.176399e-01, inlier_rmse=1.827909e-02, and correspondence_set size of 192313
Access transformation to get result.
Trying with distance threshold

In [8]:
trans=result_icp.transformation

In [13]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation, source_path)

3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=6.520412e-01, inlier_rmse=1.940616e-02, and correspondence_set size of 1709
Access transformation to get result.
[30, 0.02, 1]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=5.057555e-01, inlier_rmse=1.007513e-02, and correspondence_set size of 3471
Access transformation to get result.
[14, 0.01, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=3.751262e-01, inlier_rmse=5.228980e-03, and correspondence_set size of 6690
Access transformation to get result.
save  0013_transformed_colored_icp.ply


In [10]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017

transformed_source = copy.deepcopy(source)
transformed_source.transform(trans)

max_iter = [500, 30, 14]
voxel_radius = [0.04, 0.02, 0.01]
current_transformation = result_icp.transformation
print("3. Colored point cloud registration")

for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    # 使用する法線は元のPLYファイルから取得
    print("3-1. Estimate normal if not already done.")


    print("3-2. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        transformed_source, target, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)

draw_registration_result_original_color(transformed_source, target,
                                        result_icp.transformation, source_path)


3. Colored point cloud registration
[500, 0.04, 0]
3-1. Estimate normal if not already done.
3-2. Applying colored point cloud registration
RegistrationResult with fitness=4.906186e-01, inlier_rmse=1.861955e-02, and correspondence_set size of 116727
Access transformation to get result.
[30, 0.02, 1]
3-1. Estimate normal if not already done.
3-2. Applying colored point cloud registration
RegistrationResult with fitness=3.140620e-01, inlier_rmse=1.077780e-02, and correspondence_set size of 74721
Access transformation to get result.
[14, 0.01, 2]
3-1. Estimate normal if not already done.
3-2. Applying colored point cloud registration
RegistrationResult with fitness=1.054103e-01, inlier_rmse=6.233110e-03, and correspondence_set size of 25079
Access transformation to get result.
save  0012_norm_colored_icp.ply


In [9]:


# 3. source に変換 trans を適用し、新しい点群として保存
transformed_source = copy.deepcopy(source)
transformed_source.transform(trans)

# 初期変換を適用した点群を可視化
o3d.visualization.draw_geometries([transformed_source, target], window_name="Initial Alignment")

# 4. 通常のICPを適用
threshold = 0.1  # ICPのしきい値を設定
initial_transformation = np.identity(4)
result_icp = o3d.pipelines.registration.registration_icp(
    transformed_source, target, threshold, initial_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# 5. 結果を表示
print("ICP result:", result_icp)
print("Transformation matrix:", result_icp.transformation)

# 6. ICP後の位置合わせ結果を可視化
aligned_source = copy.deepcopy(transformed_source)
aligned_source.transform(result_icp.transformation)

o3d.visualization.draw_geometries([aligned_source, target], window_name="ICP Alignment")


ICP result: RegistrationResult with fitness=7.778394e-01, inlier_rmse=4.234602e-02, and correspondence_set size of 185062
Access transformation to get result.
Transformation matrix: [[ 9.99980802e-01 -7.45034203e-04 -6.15150423e-03  7.81254183e-03]
 [ 9.77048848e-04  9.99284836e-01  3.78003007e-02 -2.66583505e-02]
 [ 6.11894238e-03 -3.78055853e-02  9.99266379e-01  9.93331776e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [6]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])


def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    source = o3d.io.read_point_cloud(source_path)
    target = o3d.io.read_point_cloud(target_path)
    # trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
    #                          [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [7]:
voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [8]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [9]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
RegistrationResult with fitness=8.098481e-01, inlier_rmse=3.646758e-02, and correspondence_set size of 1546
Access transformation to get result.


In [10]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result



In [11]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)



:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.020.


RuntimeError: [1;31m[Open3D Error] (open3d::pipelines::registration::RegistrationResult open3d::pipelines::registration::RegistrationICP(const open3d::geometry::PointCloud&, const open3d::geometry::PointCloud&, double, const Matrix4d&, const open3d::pipelines::registration::TransformationEstimation&, const open3d::pipelines::registration::ICPConvergenceCriteria&)) /root/Open3D/cpp/open3d/pipelines/registration/Registration.cpp:128: TransformationEstimationPointToPlane and TransformationEstimationColoredICP require pre-computed normal vectors for target PointCloud.
[0;m